# 필요 라이브러리를 불러 옵니다

In [ ]:
import os
BASE_PATH = os.path.dirname(os.getcwd())
os.chdir(BASE_PATH)

In [ ]:
import tensorflow as tf

In [ ]:
from utils.datagenerator import TightFaceProvider
from utils.helper import show_images, cropper, glob_all_files, paths2numpy, draw_rectangles
import numpy as np 

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model

# 학습 시킨 모델을 불러 옵니다

In [ ]:
# 모델을 불러옵니다. 
model = load_model('./models/best_model')
model.summary()

# 월리를 찾아 봅니다.

In [ ]:
import matplotlib.pyplot as plt 
import cv2

# Validation 이미지가 담겨져 있는 폴더에 있는 모든 이미지 리스트를 가져옵니다. 
paths = glob_all_files('./data/full_images_val/')

# 이미지를 Numpy 화 합니다. 
imgs = paths2numpy(paths)

# 이미지를 일정 크기로 자르고 해당 이미지의 좌표를 가져옵니다.
crop_imgs = []
crop_crds = []
for img in imgs:
    cropped_images, cropped_coords = cropper(img, 10, 10, 34, 34)
    crop_imgs.append(cropped_images)
    crop_crds.append(cropped_coords)

# 잘린 이미지중 월리가있는 이미지를 찾고 해당이미지의 좌표를 원본 이미지에 출력합니다 
for img_ind, im in enumerate(imgs):
    cropped_imgs = crop_imgs[img_ind]
    cropped_crds = crop_crds[img_ind]
    
    # Wally 라고 생각되는 이미지의 index을 가져옵니다. 
    indices = (model.predict(cropped_imgs) > 0.5)[:, 0]
    
    # Wally 라고 생각되는 이미지 좌표를 가져옵니다.
    target_crds = np.array(cropped_crds)[indices]

    # 전체 이미지에 월리라고 예측 되는 부분에 사각형을 그립니다. 
    im = draw_rectangles(im, target_crds, (255, 0, 0), 3)
    
    fig = plt.figure(figsize=(30,30))
    plt.imshow(im)
    plt.show()
